In [2]:
import json
import re

In [3]:
from pycocotools.coco import COCO

# Initialize COCO API for caption annotations
annFile = '../ProbVLM/Datasets/coco/captions_train2014_extra.json'  # Path to COCO annotations file
coco = COCO(annFile)

# Get all available annotation IDs
all_annotation_ids = set(coco.getAnnIds())
sorted_ann_id = sorted(all_annotation_ids)

# Select a valid annotation ID (first one as an example)
valid_annotation_id = sorted_ann_id[-1]  # This should be a valid ID

# Load and print the annotation for the selected ID
annotation = coco.loadAnns(valid_annotation_id)
print("Annotation for ID", valid_annotation_id, ":", annotation)

loading annotations into memory...
Done (t=0.73s)
creating index...
index created!
Annotation for ID 874893 : [{'image_id': 262136, 'id': 874893, 'caption': 'a photo of a person'}]


In [4]:
coco2 = COCO('../ProbVLM/Datasets/coco/captions_val2014_extra.json')

loading annotations into memory...
Done (t=0.49s)
creating index...
index created!


In [20]:
path = '../ProbVLM/Datasets/coco/filtered_ids_train.txt'
path2 = '../ProbVLM/Datasets/coco/filtered_ids_val3.txt'
path3 = '../ProbVLM/Datasets/coco/filtered_ids_test3.txt'
# Step 2: Load IDs from the text file
with open(path, 'r') as file:
    filtered_cap_ids_train = [int(line.strip()) for line in file]  # Read and convert IDs to integers
    
with open(path2, 'r') as file:
    filtered_cap_ids_val = [int(line.strip()) for line in file]  # Read and convert IDs to integers
    
with open(path3, 'r') as file:
    filtered_cap_ids_test = [int(line.strip()) for line in file]  # Read and convert IDs to integers


In [21]:
len(list(filtered_cap_ids_test))

13135

In [24]:
img_ids_train = coco.loadAnns(filtered_cap_ids_train)
img_ids_val = coco2.loadAnns(filtered_cap_ids_val)
img_ids_test = coco2.loadAnns(filtered_cap_ids_test)

In [25]:
img_ids_train_1 = set([item['image_id'] for item in img_ids_train])
img_ids_val_1 = set([item['image_id'] for item in img_ids_val])
img_ids_test_1 = set([item['image_id'] for item in img_ids_test])

In [28]:
txt_file_path = '../GALS/data/COCO/COCO_gender/biased_split/test.ids.txt'
txt_file_path2 = '../GALS/data/COCO/COCO_gender/balanced_split/val_woman.txt'
txt_file_path3 = '../GALS/data/COCO/COCO_gender/balanced_split/val_man.txt'
# Step 2: Load IDs from the text file
with open(txt_file_path, 'r') as file:
    text_file_img_ids = {int(line.strip()) for line in file}  # Read and convert IDs to integers
    
with open(txt_file_path2, 'r') as file:
    text_file_img_ids2 = {int(line.strip()) for line in file}  # Read and convert IDs to integers
    
with open(txt_file_path3, 'r') as file:
    text_file_img_ids3 = {int(line.strip()) for line in file}  # Read and convert IDs to integers
    
combined_img_ids = text_file_img_ids3.union(text_file_img_ids2)

In [31]:
len(list(filtered_img_ids_test))#-500

668

In [30]:
filtered_img_ids_test = img_ids_test_1.intersection(text_file_img_ids)

In [10]:
filtered_img_ids = img_ids_val_1.intersection(text_file_img_ids)

In [100]:
filtered_img_ids_train = img_ids_train_1.intersection(text_file_img_ids)

In [32]:
def contains_synonym(caption, synonym_list):
    # Convert caption to lowercase
    caption = caption.lower()

    
    # Use regex to match whole words
    return any(re.search(r'\b' + re.escape(word) + r'\b', caption) for word in synonym_list)

woman_word_list_synonyms = ['girl', 'sister', 'mom', 'wife', 'woman',      \
        'bride', 'female', 'lady',  'actress', 'nun', 'girlfriend',        \
        'her', 'she', 'mother', 'daughter', 'businesswoman', 'cowgirl']


In [33]:
man_word_list_synonyms = ['boy', 'brother', 'dad', 'husband', 'man',       \
        'groom', 'male','guy', 'dude', 'policeman', 'father',              \
        'son', 'fireman', 'actor','gentleman', 'boyfriend',                \
        'mans', 'his', 'obama', 'businessman', 'he', 'cowboy']

In [34]:
# Initialize counters for woman and man synonyms
image_results = {}
woman_counter = 0
man_counter = 0
woman_imgs = []
man_imgs = []

# Iterate through all the image IDs and extract captions
for img_id in list(filtered_img_ids_test):  # Removed the [:1] so all images are processed
    skip_image = False  # Flag to skip the current image if both types of synonyms are found
    
    # Get annotation IDs for the current image_id
    ann_ids = coco2.getAnnIds(imgIds=img_id)
    
    # Load the annotations for the current annotation IDs
    annotations = coco2.loadAnns(ann_ids)
    
    woman_found = False
    man_found = False
    caption = None
    
    # Iterate through the annotations and extract captions
    for annotation in annotations:
        caption = annotation['caption']
        #print(f"Image ID {img_id}: {caption}")  # Print each caption
        
        # Check if the caption contains any woman synonym
        if contains_synonym(caption, woman_word_list_synonyms):
            woman_found = True

        
        # Check if the caption contains any man synonym
        if contains_synonym(caption, man_word_list_synonyms):
            man_found = True

        
        # If both woman and man synonyms are found in the same image, skip to next image
        if woman_found and man_found:

            skip_image = True
            break
    
    if not skip_image:
        if woman_found:
            woman_counter += 1
            woman_imgs.append(img_id)
        elif man_found:
            man_counter += 1
            man_imgs.append(img_id)

print('num_woman: ', woman_counter)
print('num_man: ', man_counter)


num_woman:  188
num_man:  461


In [35]:
woman_imgs

[124930,
 241668,
 28688,
 405531,
 415770,
 114744,
 376891,
 536653,
 84060,
 69757,
 436350,
 98434,
 16521,
 356505,
 522427,
 532695,
 383211,
 424172,
 545007,
 6393,
 354559,
 286981,
 4359,
 211206,
 504074,
 192782,
 489763,
 39202,
 477483,
 168243,
 491835,
 516416,
 479562,
 561491,
 133470,
 29045,
 459141,
 377239,
 192932,
 551338,
 170436,
 61899,
 504293,
 250344,
 20992,
 227851,
 49688,
 127516,
 457249,
 211491,
 105011,
 10825,
 279149,
 469618,
 209527,
 156282,
 266892,
 234153,
 242365,
 164548,
 64196,
 21198,
 563927,
 47837,
 482021,
 199404,
 115455,
 109316,
 488198,
 99081,
 213773,
 152336,
 217872,
 70426,
 346934,
 553788,
 572226,
 11099,
 258911,
 451431,
 525170,
 127865,
 355197,
 426878,
 326555,
 476065,
 91045,
 201637,
 504747,
 11181,
 465862,
 357322,
 510955,
 216051,
 209917,
 252927,
 5123,
 580613,
 85007,
 424975,
 324638,
 125983,
 205866,
 283698,
 58425,
 400453,
 422998,
 56405,
 246876,
 230501,
 128140,
 472216,
 195750,
 580778,
 5

In [124]:

# Initialize counters for woman and man synonyms
image_results = {}
woman_counter = 0
man_counter = 0
#woman_imgs_ = []
#man_imgs = []

# Iterate through all the image IDs and extract captions
for img_id in chunck_val:  # Removed the [:1] so all images are processed
    skip_image = False  # Flag to skip the current image if both types of synonyms are found
    
    # Get annotation IDs for the current image_id
    ann_ids = coco.getAnnIds(imgIds=img_id)
    
    # Load the annotations for the current annotation IDs
    annotations = coco.loadAnns(ann_ids)
    
    woman_found = False
    man_found = False
    caption = None
    
    # Iterate through the annotations and extract captions
    for annotation in annotations:
        caption = annotation['caption']
        #print(f"Image ID {img_id}: {caption}")  # Print each caption
        
        # Check if the caption contains any woman synonym
        if contains_synonym(caption, woman_word_list_synonyms):
            woman_found = True

        
        # Check if the caption contains any man synonym
        if contains_synonym(caption, man_word_list_synonyms):
            man_found = True

        
        # If both woman and man synonyms are found in the same image, skip to next image
        if woman_found and man_found:

            skip_image = True
            break
    
    if not skip_image:
        if woman_found:
            woman_counter += 1
            woman_imgs.append(img_id)
        elif man_found:
            man_counter += 1
            man_imgs.append(img_id)

print('num_woman: ', woman_counter)
print('num_man: ', man_counter)


num_woman:  117
num_man:  366


In [36]:
woman_imgs_final = woman_imgs[:188] 
man_imgs_final = man_imgs[:188] 

In [38]:
file_path = '../GALS/data/COCO/COCO_gender/balanced_split/test_man_imgs_final.txt'

# Save the list to the text file
with open(file_path, 'w') as file:
    for img_id in man_imgs_final:
        file.write(f"{img_id}\n")